In [1]:
# pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [5]:
# !pip install tensorflow

In [6]:
# pip install distutils

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement distutils (from versions: none)
ERROR: No matching distribution found for distutils


In [1]:
# pip uninstall tensorflow


^C
Note: you may need to restart the kernel to use updated packages.


In [10]:
# pip install tensorflow


  Using cached tensorflow-2.16.1-cp312-cp312-win_amd64.whl.metadata (3.5 kB)
  Using cached tensorflow_intel-2.16.1-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
Using cached tensorflow-2.16.1-cp312-cp312-win_amd64.whl (2.1 kB)
Using cached tensorflow_intel-2.16.1-cp312-cp312-win_amd64.whl (377.1 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device



In [2]:
import tensorflow as tf
print(tf.__version__)

2.16.1


In [3]:
import tensorflow as tf
print(tf.__version__)

2.16.1


In [4]:
import tensorflow as tf
import tensorflow.keras.layers as L
# import tensorflow_addons as tfa
import os,warnings

In [5]:
image_size=224

In [6]:
import kagglehub
# Download latest version
path = kagglehub.model_download("spsayakpaul/vision-transformer/tensorFlow2/vit-b16-classification")
print("Path to model files:", path)


Path to model files: C:\Users\mouli\.cache\kagglehub\models\spsayakpaul\vision-transformer\tensorFlow2\vit-b16-classification\1


In [7]:
# pip install tensorflow_hub

Note: you may need to restart the kernel to use updated packages.


In [7]:
import tensorflow as tf
import tensorflow_hub as hub
# Load ViT-B16 model from TensorFlow Hub
vit_model = hub.KerasLayer(path, trainable=True ,input_shape=(224,224,3))

In [8]:
vit_lambda = tf.keras.layers.Lambda(lambda x: vit_model(x))
# predictions = vit_model(input_images_normalized)

In [9]:
model = tf.keras.Sequential([
        vit_lambda,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1000, activation="relu")
        ],
        name = 'vision_transformer')

In [10]:
def extract_vit_features(model, dataset):
    features=model.predict(dataset)
    return np.vstack(features)
# vit_features=extract_vit_features(model, data)

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import ResNet50
def create_resnet_model(input_shape):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    model = Model(inputs=base_model.input, outputs=x)
    return model
def extract_resnet_features(model, dataset):
    features=model.predict(dataset)
    return np.vstack(features)
res_model=create_resnet_model((224,224,3))
# res_features=extract_resnet_features(res_model, data)


In [12]:
l=[
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___healthy',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Grape___Black_rot',
 'Grape___healthy',
 'Grape__Leaf_blight(Isariopsis_Leaf_Spot)',
 'Potato___Early_blight',
 'Potato___healthy',
 'Potato___Late_blight',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___healthy'
]
d={i:j for i,j in enumerate(l)}

In [13]:
d.items()

dict_items([(0, 'Apple___Black_rot'), (1, 'Apple___Cedar_apple_rust'), (2, 'Apple___healthy'), (3, 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot'), (4, 'Corn_(maize)___healthy'), (5, 'Corn_(maize)___Northern_Leaf_Blight'), (6, 'Grape___Black_rot'), (7, 'Grape___healthy'), (8, 'Grape__Leaf_blight(Isariopsis_Leaf_Spot)'), (9, 'Potato___Early_blight'), (10, 'Potato___healthy'), (11, 'Potato___Late_blight'), (12, 'Tomato___Bacterial_spot'), (13, 'Tomato___Early_blight'), (14, 'Tomato___healthy')])

In [18]:
from flask import Flask, render_template, request
from PIL import Image
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model 
from keras.applications import Xception
from keras.models import Model
import pickle
import keras
app = Flask(__name__)
from keras.models import load_model
model2=load_model( r'b6vit_resnet_model3.keras')
def preprocess_image(image, target_size):
    if image.mode != "RGB":
        image = image.convert("RGB")
    image = image.resize(target_size)
    image = np.array(image)
    image = np.expand_dims(image, axis=0)
    return image
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        print("innnnnnnnnnnnnnnnn")
        file = request.files['file']
        image = Image.open(file)
        img = preprocess_image(image, target_size=(224, 224))
        img=np.reshape(img,(1,224,224,3))
        print('image shape:\n\n',img.shape) 
        res_feature=extract_resnet_features(res_model, img)
        vit_feature=extract_vit_features(model, img)
        bottleneck_features = layers.Concatenate()([res_feature, vit_feature])
        preds=model2.predict(bottleneck_features)
        labls=np.argmax(preds)
        return d[labls]+'----'+str(labls)
if __name__ == '_main_':
    app.run()

In [23]:
from flask import Flask, render_template, request
from PIL import Image
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model 
from keras.applications import Xception
from keras.models import Model
import pickle
import keras
app = Flask(__name__)
from keras.models import load_model
model2=load_model( r'b6vit_resnet_model3.keras')
@app.route('/')
def index():
    return render_template('index1.html')

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        print("innnnnnnnnnnnnnnnn")
        file = request.files['file']
        img = Image.open(file.stream)
        img = image.img_to_array(img)
        img=np.reshape(img,(1,224,224,3))
        print('image shape:\n\n',img.shape) 
        res_feature=extract_resnet_features(res_model, img)
        vit_feature=extract_vit_features(model, img)
        bottleneck_features = layers.Concatenate()([res_feature, vit_feature])
        preds=model2.predict(bottleneck_features)
        labls=np.argmax(preds)
        
        return d[labls]+'----'+str(labls)
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Jun/2024 22:55:41] "GET / HTTP/1.1" 200 -


innnnnnnnnnnnnnnnn
image shape:

 (1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 516ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step


INFO:werkzeug:127.0.0.1 - - [02/Jun/2024 22:55:47] "POST /predict HTTP/1.1" 200 -
ERROR:__main__:Exception on /predict [POST]
Traceback (most recent call last):
  File "c:\Users\mouli\anaconda3\Lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mouli\anaconda3\Lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mouli\anaconda3\Lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mouli\anaconda3\Lib\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mouli\AppData\Local\Temp\ipykernel_15000\27737

innnnnnnnnnnnnnnnn


...

innnnnnnnnnnnnnnnn
image shape:

 (1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


INFO:werkzeug:127.0.0.1 - - [02/Jun/2024 22:57:29] "POST /predict HTTP/1.1" 200 -
ERROR:__main__:Exception on /predict [POST]
Traceback (most recent call last):
  File "c:\Users\mouli\anaconda3\Lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mouli\anaconda3\Lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mouli\anaconda3\Lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mouli\anaconda3\Lib\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mouli\AppData\Local\Temp\ipykernel_15000\27737

innnnnnnnnnnnnnnnn
innnnnnnnnnnnnnnnn
image shape:

 (1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 567ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


INFO:werkzeug:127.0.0.1 - - [02/Jun/2024 22:59:47] "POST /predict HTTP/1.1" 200 -
ERROR:__main__:Exception on /predict [POST]
Traceback (most recent call last):
  File "c:\Users\mouli\anaconda3\Lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mouli\anaconda3\Lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mouli\anaconda3\Lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mouli\anaconda3\Lib\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mouli\AppData\Local\Temp\ipykernel_15000\27737

innnnnnnnnnnnnnnnn
